<p align="center">
<img src="Images/sorbonne_logo.png" alt="Logo" width="300"/>
</p>

# **Module 1 - Data Extraction & Manipulation**

* **Author**: Elia Landini
* **Student ID**: 12310239
* **Course**: EESM2-Financial Economics 
* **Supervisor**: XXXXXXXXXXXXXXXX
* **Reference Repository**: https://github.com/EliaLand/PVAR_japan_endogenous_money

### **1) PREFACE**

abcd

### **2) REQUIREMENTS SET-UP**

In [1]:
# Requirements.txt file installation
# !pip install -r requirements.txt

In [2]:
# Libraries import
import warnings
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import levene
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller
import sklearn.tree
import sklearn.metrics
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing 
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             precision_score, recall_score, f1_score,
                             accuracy_score, precision_recall_curve, auc, 
                             RocCurveDisplay, ConfusionMatrixDisplay)
from sklearn.linear_model import (LinearRegression, LogisticRegression)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight
import plotly.express as px
import openpyxl as pxl
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
from IPython.display import Image
import itertools
from arch.unitroot import PhillipsPerron

### **3) HELPER FUNCTIONS & GENERAL VARIABLES**

In [3]:
# Custom modules import
from Modules.FRED_module import fetch_FRED
from Modules.EUROSTAT_module import fetch_EUROSTAT
from Modules.WB_module import fetch_WB 
from Modules.YFINANCE_module import fetch_YFINANCE
from Modules.DBNOMICS_module import fetch_DBNOMICS

In [4]:
# Statistical Significance labelling 
def significance_stars(p):
    if p < 0.001:
        return "***"  
    elif p < 0.01:
        return "**"    
    elif p < 0.05:
        return "*"   
    else:
        return ""

In [5]:
# We supress potential warnings with this command
warnings.filterwarnings("ignore")

### **4) DATA RETRIEVAL**

##### **4.1) REAL EFFECTIVE EXCHANGE RATES**

In [6]:
# Real Effective Exchange Rates: CPI Based for Japan (monthly, Index 2015=100, non-seasonally adjusted, 1970-01, 2025-09)
# https://fred.stlouisfed.org/series/CCRETT01JPM661N

REXUSDJPY_m_raw = fetch_FRED("CCRETT01JPM661N") 
REXUSDJPY_m_raw = REXUSDJPY_m_raw.rename(columns= 
        {"date": "Time", 
         "CCRETT01JPM661N": "USD-JPY reer CPI-based (Index 2015=100)"
})

REXUSDJPY_m_raw["Time"] = REXUSDJPY_m_raw["Time"].dt.to_period("M").astype(str)

REXUSDJPY_m_raw["Country"] = "JP"
REXUSDJPY_m_raw = REXUSDJPY_m_raw[["Country", "Time", "USD-JPY reer CPI-based (Index 2015=100)"]]

REXUSDJPY_m_raw.tail()

,Country,Time,USD-JPY reer CPI-based (Index 2015=100)
666,JP,2025-07,81.29183
667,JP,2025-08,81.26475
668,JP,2025-09,80.87212
669,JP,2025-10,79.31219
670,JP,2025-11,77.64403


##### **4.2) CONSUMER PRICES (HICP), ALL ITEMS (WHOLE COUNTRY), NSA**

In [7]:
# Consumer prices (HICP), all items (whole country), NSA, Japan (monthly, Index April 2025=111.5, non-seasonally adjusted, 1970-01, 2025-04)
# https://data.ecb.europa.eu/data/datasets/RTD/RTD.M.JP.N.P_C_OV.X

jp_HICP_m_raw = pd.read_csv("Data/HICP_ECB_extracted_raw.csv") 
jp_HICP_m_raw = jp_HICP_m_raw.drop(columns=["DATE"])
jp_HICP_m_raw = jp_HICP_m_raw.rename(columns= 
        {"TIME PERIOD": "Time", 
         "CONSUMER PRICES, ALL ITEMS (WHOLE COUNTRY), NSA (RTD.M.JP.N.P_C_OV.X)": "HICP (NSA) - Japan"
})     
                          
jp_HICP_m_raw["Time"] = pd.to_datetime(jp_HICP_m_raw["Time"], format="%Y%b", errors="coerce")
jp_HICP_m_raw["Time"] = jp_HICP_m_raw["Time"].dt.to_period("M").astype(str)

jp_HICP_m_raw["Country"] = "JP"
jp_HICP_m_raw = jp_HICP_m_raw[["Country", "Time", "HICP (NSA) - Japan"]]

jp_HICP_m_raw.tail()

,Country,Time,HICP (NSA) - Japan
659,JP,2024-12,110.7
660,JP,2025-01,111.2
661,JP,2025-02,110.8
662,JP,2025-03,111.1
663,JP,2025-04,111.5


##### **4.3) JPY-USD SPOT EXCHANGE RATE**

In [8]:
# Japanese Yen to U.S. Dollar Spot Exchange Rate (monthly, non-seasonally adjusted, 1971-01, 2025-09)
# https://fred.stlouisfed.org/series/EXJPUS

JPYUSD_m_raw = fetch_FRED("EXJPUS") 
JPYUSD_m_raw = JPYUSD_m_raw.rename(columns= 
        {"date": "Time", 
         "EXJPUS": "JPY-USD Spot Exchange Rate"
})

JPYUSD_m_raw["Time"] = JPYUSD_m_raw["Time"].dt.to_period("M").astype(str)

JPYUSD_m_raw["Country"] = "JP"
JPYUSD_m_raw = JPYUSD_m_raw[["Country", "Time", "JPY-USD Spot Exchange Rate"]]

JPYUSD_m_raw.tail()

,Country,Time,JPY-USD Spot Exchange Rate
655,JP,2025-08,147.4786
656,JP,2025-09,147.8629
657,JP,2025-10,151.3545
658,JP,2025-11,155.1411
659,JP,2025-12,155.9150


##### **4.4) JPY-USD SPOT EXCHANGE RATE**

In [9]:
# Monthly price and volume of Japan's stock indeces and bond-related instruments (monthly, log, number of securities traded, 2015-01, 2025-10)
# (!!!) Volume column is dangerous, a lot of 0 values, depending on the index, it must be carefully handled 

jp_stock_indices_tickers = {
    "Nikkei 225": "^N225",
    "NYSE Arca Japan Index": "^JPN",
    "iShares 7‑10 Year Japan Government Bond ETF": "236A.T",
    "iShares Core Japan Government Bond ETF": "2561.T"
}

start = "2015-01-01"
end = "2025-10-25"

list_single_country_dfs = []

# We iterate over each country and respective stock index
# We aggregate data through concatenation based on y axis
for index, ticker in jp_stock_indices_tickers.items():
    df = fetch_YFINANCE(ticker, start, end)

# MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns]
    df = df.rename(columns={
        "Close": "Index-specific Closing Price",
        "YearMonth": "Time"
    })
    df["Log Monthly Return"] = np.log(df["Index-specific Closing Price"] / df["Index-specific Closing Price"].shift(1))
    df["Stock Index / Bond-related Instrument"] = index
    df["Stock Index"] = ticker

    df = df[["Stock Index / Bond-related Instrument", "Stock Index", "Time", "Log Monthly Return", "Volume"]]

    list_single_country_dfs.append(df)

jp_stock_m_raw = pd.concat(list_single_country_dfs, ignore_index=True)

jp_stock_m_raw.tail()

,Stock Index / Bond-related Instrument,Stock Index,Time,Log Monthly Return,Volume
328,iShares Core Japan Government Bond ETF,2561.T,2025-06,0.007204,282937
329,iShares Core Japan Government Bond ETF,2561.T,2025-07,-0.016282,276779
330,iShares Core Japan Government Bond ETF,2561.T,2025-08,-0.004130,178697
331,iShares Core Japan Government Bond ETF,2561.T,2025-09,0.003209,273959
332,iShares Core Japan Government Bond ETF,2561.T,2025-10,-0.000916,691174


##### **4.5) TOTAL RESERVES EXCLUDING GOLD (JPN)**

In [10]:
# Total Reserves excluding Gold for Japan (monthly, millions USD, non-seasonally adjusted, 1950-01, 2025-07)
# https://fred.stlouisfed.org/series/TRESEGJPM052N 

jp_reserves_m_raw = fetch_FRED("TRESEGJPM052N") 
jp_reserves_m_raw = jp_reserves_m_raw.rename(columns= 
        {"date": "Time", 
         "TRESEGJPM052N": "Total Treasury Reserves (- Gold)"
})

jp_reserves_m_raw["Time"] = jp_reserves_m_raw["Time"].dt.to_period("M").astype(str)

jp_reserves_m_raw["Country"] = "JP"
jp_reserves_m_raw = jp_reserves_m_raw[["Country", "Time", "Total Treasury Reserves (- Gold)"]]

jp_reserves_m_raw.tail()

,Country,Time,Total Treasury Reserves (- Gold)
895,JP,2025-07,1.214711e+06
896,JP,2025-08,1.230941e+06
897,JP,2025-09,1.237223e+06
898,JP,2025-10,1.238269e+06
899,JP,2025-11,1.245355e+06


##### **4.6) MONETARY AGGREGATES - BROAD MONEY & COMPONENTS - M3 for Japan**

In [11]:
# Monetary Aggregates and Their Components: Broad Money and Components: M3 for Japan (monthly, YPJ , seasonally adjusted, 1980-01, 2023-11)
# https://fred.stlouisfed.org/series/MABMM301JPM189S

jp_m3_m_raw = fetch_FRED("MABMM301JPM189S") 
jp_m3_m_raw = jp_m3_m_raw.rename(columns= 
        {"date": "Time", 
         "MABMM301JPM189S": "Monetary Aggregates - M3 (JPY)"
})

jp_m3_m_raw["Time"] = jp_m3_m_raw["Time"].dt.to_period("M").astype(str)

jp_m3_m_raw["Country"] = "JP"
jp_m3_m_raw = jp_m3_m_raw[["Country", "Time", "Monetary Aggregates - M3 (JPY)"]]

jp_m3_m_raw.tail()

,Country,Time,Monetary Aggregates - M3 (JPY)
522,JP,2023-07,1.588088e+15
523,JP,2023-08,1.590559e+15
524,JP,2023-09,1.592132e+15
525,JP,2023-10,1.595186e+15
526,JP,2023-11,1.597004e+15


##### **4.7) MONETARY AGGREGATES - BROAD MONEY & COMPONENTS - M2 for Japan**

In [12]:
# Monetary Aggregates and Their Components: Broad Money and Components: M2 for Japan (monthly, YPJ, seasonally adjusted, 1955-01, 2013-12)
# https://fred.stlouisfed.org/series/MABMM201JPM189S

jp_m2_m_raw = fetch_FRED("MABMM201JPM189S") 
jp_m2_m_raw = jp_m2_m_raw.rename(columns= 
        {"date": "Time", 
         "MABMM201JPM189S": "Monetary Aggregates - M2 (JPY)"
})

jp_m2_m_raw["Time"] = jp_m2_m_raw["Time"].dt.to_period("M").astype(str)

jp_m2_m_raw["Country"] = "JP"
jp_m2_m_raw = jp_m2_m_raw[["Country", "Time", "Monetary Aggregates - M2 (JPY)"]]

jp_m2_m_raw.tail()

,Country,Time,Monetary Aggregates - M2 (JPY)
703,JP,2013-08,8.499351e+14
704,JP,2013-09,8.528763e+14
705,JP,2013-10,8.564095e+14
706,JP,2013-11,8.595146e+14
707,JP,2013-12,8.628461e+14


##### **4.8) TOTAL CREDIT TO PRIVATE NON-FINANCIAL SECTOR**

In [13]:
# Total Credit to Private Non-Financial Sector, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1964-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPPAM770A

jp_credit_pnf_m_raw = fetch_FRED("QJPPAM770A") 
jp_credit_pnf_m_raw = jp_credit_pnf_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPPAM770A": "Total Credit - Private Non-Financial (%GDP)"
})
jp_credit_pnf_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_pnf_m_raw["Time"] = pd.PeriodIndex(jp_credit_pnf_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_pnf_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - Private Non-Financial (%GDP)": row["Total Credit - Private Non-Financial (%GDP)"] 
        })

jp_credit_pnf_m_raw = pd.DataFrame(expanded_rows)
jp_credit_pnf_m_raw["Country"] = "JP"
jp_credit_pnf_m_raw = jp_credit_pnf_m_raw[["Country", "Time", "Total Credit - Private Non-Financial (%GDP)"]]

jp_credit_pnf_m_raw.tail()

,Country,Time,Total Credit - Private Non-Financial (%GDP)
724,JP,2025-02,180.1
725,JP,2025-03,180.1
726,JP,2025-04,179.3
727,JP,2025-05,179.3
728,JP,2025-06,179.3


##### **4.9) TOTAL CREDIT TO GENERAL GOVERNMENT**

In [14]:
# Total Credit to General Government, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1997-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPGAM770A

jp_credit_gg_m_raw = fetch_FRED("QJPGAM770A") 
jp_credit_gg_m_raw = jp_credit_gg_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPGAM770A": "Total Credit - General Government (%GDP)"
})
jp_credit_gg_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_gg_m_raw["Time"] = pd.PeriodIndex(jp_credit_gg_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_gg_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - General Government (%GDP)": row["Total Credit - General Government (%GDP)"] 
        })

jp_credit_gg_m_raw = pd.DataFrame(expanded_rows)
jp_credit_gg_m_raw["Country"] = "JP"
jp_credit_gg_m_raw = jp_credit_gg_m_raw[["Country", "Time", "Total Credit - General Government (%GDP)"]]

jp_credit_gg_m_raw.tail()

,Country,Time,Total Credit - General Government (%GDP)
328,JP,2025-02,200.6
329,JP,2025-03,200.6
330,JP,2025-04,198.1
331,JP,2025-05,198.1
332,JP,2025-06,198.1


##### **4.10) TOTAL CREDIT TO HOUSEHOLDS & NPISHs**

In [15]:
# Total Credit to households and NPISHs, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1964-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPHAM770A

jp_credit_households_m_raw = fetch_FRED("QJPHAM770A") 
jp_credit_households_m_raw = jp_credit_households_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPHAM770A": "Total Credit - Households & NPISHs (%GDP)"
})
jp_credit_households_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_households_m_raw["Time"] = pd.PeriodIndex(jp_credit_households_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_households_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - Households & NPISHs (%GDP)": row["Total Credit - Households & NPISHs (%GDP)"] 
        })

jp_credit_households_m_raw = pd.DataFrame(expanded_rows)
jp_credit_households_m_raw["Country"] = "JP"
jp_credit_households_m_raw = jp_credit_households_m_raw[["Country", "Time", "Total Credit - Households & NPISHs (%GDP)"]]

jp_credit_households_m_raw.tail()

,Country,Time,Total Credit - Households & NPISHs (%GDP)
724,JP,2025-02,64.5
725,JP,2025-03,64.5
726,JP,2025-04,64.0
727,JP,2025-05,64.0
728,JP,2025-06,64.0


##### **4.11) REAL GDP**

In [16]:
# Real Gross Domestic Product for Japan (quarterly, billions of chained 2015 JPY, seasonally adjusted, 1994-01, 2025-04)
# https://fred.stlouisfed.org/series/JPNRGDPEXP

jp_rgdp_m_raw = fetch_FRED("JPNRGDPEXP") 
jp_rgdp_m_raw = jp_rgdp_m_raw.rename(columns= 
        {"date": "Time", 
         "JPNRGDPEXP": "Real GDP (billions chained 2015 JPY)"
})
jp_rgdp_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_rgdp_m_raw["Time"] = pd.PeriodIndex(jp_rgdp_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_rgdp_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Real GDP (billions chained 2015 JPY)": row["Real GDP (billions chained 2015 JPY)"] 
        })

jp_rgdp_m_raw = pd.DataFrame(expanded_rows)
jp_rgdp_m_raw["Country"] = "JP"
jp_rgdp_m_raw = jp_rgdp_m_raw[["Country", "Time", "Real GDP (billions chained 2015 JPY)"]]

jp_rgdp_m_raw.tail()

,Country,Time,Real GDP (billions chained 2015 JPY)
376,JP,2025-05,593596.9
377,JP,2025-06,593596.9
378,JP,2025-07,590141.1
379,JP,2025-08,590141.1
380,JP,2025-09,590141.1


##### **4.12) LONG-TERM GOVERNMENT BOND YIELDS - 10-YEAR**

In [17]:
# Interest Rates: Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Japan (monthly, percent, non-seasonally adjusted, 1989-01, 2025-09)
# https://fred.stlouisfed.org/series/IRLTLT01JPM156N

jp_10ygb_m_raw = fetch_FRED("IRLTLT01JPM156N") 
jp_10ygb_m_raw = jp_10ygb_m_raw.rename(columns= 
        {"date": "Time", 
         "IRLTLT01JPM156N": "10-Year Gov Bond Yields (%)"
})

jp_10ygb_m_raw["Time"] = jp_10ygb_m_raw["Time"].dt.to_period("M").astype(str)

jp_10ygb_m_raw["Country"] = "JP"
jp_10ygb_m_raw = jp_10ygb_m_raw[["Country", "Time", "10-Year Gov Bond Yields (%)"]]

jp_10ygb_m_raw.tail()

,Country,Time,10-Year Gov Bond Yields (%)
438,JP,2025-07,1.545
439,JP,2025-08,1.600
440,JP,2025-09,1.645
441,JP,2025-10,1.655
442,JP,2025-11,1.805


##### **4.13) IMMEDIATE RATES (< 24 Hours): CALL MONEY/INTERBANK RATE**

In [18]:
# Interest Rates: Immediate Rates (< 24 Hours): Call Money/Interbank Rate: Total for Japan (monthly, percent, non-seasonally adjusted, 1985-01, 2025-09)
# https://fred.stlouisfed.org/series/IRSTCI01JPM156N

jp_cmibr_m_raw = fetch_FRED("IRSTCI01JPM156N") 
jp_cmibr_m_raw = jp_cmibr_m_raw.rename(columns= 
        {"date": "Time", 
         "IRSTCI01JPM156N": "Call Money/Interbank Immediate (%)"
})

jp_cmibr_m_raw["Time"] = jp_cmibr_m_raw["Time"].dt.to_period("M").astype(str)

jp_cmibr_m_raw["Country"] = "JP"
jp_cmibr_m_raw = jp_cmibr_m_raw[["Country", "Time", "Call Money/Interbank Immediate (%)"]]

jp_cmibr_m_raw.tail()

,Country,Time,Call Money/Interbank Immediate (%)
480,JP,2025-07,0.478
481,JP,2025-08,0.477
482,JP,2025-09,0.477
483,JP,2025-10,0.477
484,JP,2025-11,0.478


### **5) DATA MERGING**

In [19]:
# Broad Money Dataframe
jp_broad_money_df = pd.merge(jp_m2_m_raw, jp_m3_m_raw, on=["Country", "Time"], how="outer")

jp_broad_money_df.to_csv("Data/Aggregated/jp_broad_money_df.csv", index=False)
jp_broad_money_df.tail()

,Country,Time,Monetary Aggregates - M2 (JPY),Monetary Aggregates - M3 (JPY)
822,JP,2023-07,NaN,1.588088e+15
823,JP,2023-08,NaN,1.590559e+15
824,JP,2023-09,NaN,1.592132e+15
825,JP,2023-10,NaN,1.595186e+15
826,JP,2023-11,NaN,1.597004e+15


In [20]:
# Credit Demand Dataframe
jp_credit_demand_df = pd.merge(jp_credit_pnf_m_raw, jp_credit_gg_m_raw, on=["Country", "Time"], how="outer")
jp_credit_demand_df = pd.merge(jp_credit_demand_df, jp_credit_households_m_raw , on=["Country", "Time"], how="outer")

jp_credit_demand_df.to_csv("Data/Aggregated/jp_credit_demand_df.csv", index=False)
jp_credit_demand_df.tail()

,Country,Time,Total Credit - Private Non-Financial (%GDP),Total Credit - General Government (%GDP),Total Credit - Households & NPISHs (%GDP)
724,JP,2025-02,180.1,200.6,64.5
725,JP,2025-03,180.1,200.6,64.5
726,JP,2025-04,179.3,198.1,64.0
727,JP,2025-05,179.3,198.1,64.0
728,JP,2025-06,179.3,198.1,64.0


In [21]:
# Bank Reserves Dataframe
jp_bank_reserves_df = jp_reserves_m_raw.copy()

jp_bank_reserves_df.to_csv("Data/Aggregated/jp_bank_reserves_df.csv", index=False)
jp_bank_reserves_df.tail()

,Country,Time,Total Treasury Reserves (- Gold)
895,JP,2025-07,1.214711e+06
896,JP,2025-08,1.230941e+06
897,JP,2025-09,1.237223e+06
898,JP,2025-10,1.238269e+06
899,JP,2025-11,1.245355e+06


In [22]:
# Policy Rate Dataframe
jp_policy_rate_df = pd.merge(jp_10ygb_m_raw, jp_cmibr_m_raw, on=["Country", "Time"], how="outer")

jp_policy_rate_df.to_csv("Data/Aggregated/jp_policy_rate_df.csv", index=False)
jp_policy_rate_df.tail()

,Country,Time,10-Year Gov Bond Yields (%),Call Money/Interbank Immediate (%)
480,JP,2025-07,1.545,0.478
481,JP,2025-08,1.600,0.477
482,JP,2025-09,1.645,0.477
483,JP,2025-10,1.655,0.477
484,JP,2025-11,1.805,0.478


In [23]:
# Exchange Rate Dataframe
jp_exchange_rate_df = pd.merge(REXUSDJPY_m_raw, JPYUSD_m_raw, on=["Country", "Time"], how="outer")

jp_exchange_rate_df.to_csv("Data/Aggregated/jp_exchange_rate_df.csv", index=False)
jp_exchange_rate_df.tail()

,Country,Time,USD-JPY reer CPI-based (Index 2015=100),JPY-USD Spot Exchange Rate
667,JP,2025-08,81.26475,147.4786
668,JP,2025-09,80.87212,147.8629
669,JP,2025-10,79.31219,151.3545
670,JP,2025-11,77.64403,155.1411
671,JP,2025-12,NaN,155.9150


In [24]:
# Inflation Dataframe
jp_inflation_df = jp_HICP_m_raw.copy()

jp_inflation_df.to_csv("Data/Aggregated/jp_inflation_df.csv", index=False)
jp_inflation_df.tail()

,Country,Time,HICP (NSA) - Japan
659,JP,2024-12,110.7
660,JP,2025-01,111.2
661,JP,2025-02,110.8
662,JP,2025-03,111.1
663,JP,2025-04,111.5


In [25]:
# Real GDP Dataframe
jp_rgdp_df = jp_rgdp_m_raw.copy()

jp_rgdp_df.to_csv("Data/Aggregated/jp_rgdp_df.csv", index=False)
jp_rgdp_df.tail()

,Country,Time,Real GDP (billions chained 2015 JPY)
376,JP,2025-05,593596.9
377,JP,2025-06,593596.9
378,JP,2025-07,590141.1
379,JP,2025-08,590141.1
380,JP,2025-09,590141.1


In [26]:
# Stock Indeces Dataframe
jp_stocks_pv_df = jp_stock_m_raw.copy()

jp_stocks_pv_df.to_csv("Data/Aggregated/jp_stocks_pv_df.csv", index=False)
jp_stocks_pv_df.tail()

,Stock Index / Bond-related Instrument,Stock Index,Time,Log Monthly Return,Volume
328,iShares Core Japan Government Bond ETF,2561.T,2025-06,0.007204,282937
329,iShares Core Japan Government Bond ETF,2561.T,2025-07,-0.016282,276779
330,iShares Core Japan Government Bond ETF,2561.T,2025-08,-0.004130,178697
331,iShares Core Japan Government Bond ETF,2561.T,2025-09,0.003209,273959
332,iShares Core Japan Government Bond ETF,2561.T,2025-10,-0.000916,691174


In [27]:
# Aggregated JP Dataframe
jp_aggregated_df = pd.merge(jp_broad_money_df, jp_credit_demand_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_bank_reserves_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_policy_rate_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_exchange_rate_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_inflation_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_rgdp_df, on=["Country", "Time"], how="outer")


jp_aggregated_df.to_csv("Data/Aggregated/jp_aggregated_df.csv", index=False)
jp_aggregated_df.tail()

,Country,Time,Monetary Aggregates - M2 (JPY),Monetary Aggregates - M3 (JPY),Total Credit - Private Non-Financial (%GDP),Total Credit - General Government (%GDP),Total Credit - Households & NPISHs (%GDP),Total Treasury Reserves (- Gold),10-Year Gov Bond Yields (%),Call Money/Interbank Immediate (%),USD-JPY reer CPI-based (Index 2015=100),JPY-USD Spot Exchange Rate,HICP (NSA) - Japan,Real GDP (billions chained 2015 JPY)
896,JP,2025-08,NaN,NaN,NaN,NaN,NaN,1.230941e+06,1.600,0.477,81.26475,147.4786,NaN,590141.1
897,JP,2025-09,NaN,NaN,NaN,NaN,NaN,1.237223e+06,1.645,0.477,80.87212,147.8629,NaN,590141.1
898,JP,2025-10,NaN,NaN,NaN,NaN,NaN,1.238269e+06,1.655,0.477,79.31219,151.3545,NaN,NaN
899,JP,2025-11,NaN,NaN,NaN,NaN,NaN,1.245355e+06,1.805,0.478,77.64403,155.1411,NaN,NaN
900,JP,2025-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.9150,NaN,NaN
